<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lets-do-some-real-work....-starting-with-the-parameters-defining-the-run." data-toc-modified-id="Lets-do-some-real-work....-starting-with-the-parameters-defining-the-run.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lets do some real work.... starting with the parameters defining the run.</a></span><ul class="toc-item"><li><span><a href="#Ok.-Lets-do-the-actual-scanning" data-toc-modified-id="Ok.-Lets-do-the-actual-scanning-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Ok. Lets do the actual scanning</a></span></li></ul></li></ul></div>

In [1]:
import requests
import time
import numpy as np
import uuid
import shutil
import yaml

Define some useful functions and the addresses of the bServer and mServer globally

In [2]:
bServer = "http://127.0.0.1:18085/SIS/"
mServer = "http://127.0.0.1:18080"

In [44]:
def create_UUID():
    """UUIDs will be unique for a given run of the kernel"""
    unique_uuid = uuid.uuid4().hex[:8].upper()
    return unique_uuid


def create_SPECnewfile(new_name):
    t0=time.time()
    
    wait_until_SPECfinished()
    
    payload = {'spec_cmd': 'newfile {}'.format(new_name)}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent

def cam_refresh():
    t0=time.time()
    
    wait_until_SPECfinished()
    
    payload = {'spec_cmd': 'camrefresh'}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    time.sleep(45)
    wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent


def take_pilatus_image(exp_time, move_image_to=None):
    wait_until_SPECfinished()
    
    
    payload = {'spec_cmd': 'loopscan 1 {}'.format(exp_time)}
    r = requests.get(bServer + "execute_command", params=payload)
    
    wait_until_SPECfinished()

    counters = read_counters()
    
    image_savedpath = None
    if move_image_to is not None:
        pass
        
    
    return counters

def move_SPECmotor(motor_name, value, wait_until_finished = True):
    t0=time.time()
    payload = {'spec_cmd': 'mv {} {}'.format(motor_name, value)}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    if wait_until_finished is True:
        wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent

def set_SPECshutter(shutterOpen):
    t0=time.time()
    if shutterOpen is True:
        payload = {'spec_cmd': 'sopen'}
    else:
        payload = {'spec_cmd': 'sclose'}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    time.sleep(0.1)
    
    time_spent = time.time() - t0
    return time_spent

def wait_until_SPECfinished():
    t0 = time.time()
    while True:
        r = requests.get(bServer + "is_busy")
        is_busy = r.json()['data']
            
        if is_busy is False:
            break
    return time.time() - t0

def read_counters(save_to_file=None):
    t0 = time.time()
    
    #Get the mnemonics - a little wasteful but that is ok I think right now.
    r = requests.get(bServer + "get_all_counter_mnemonics", params={})
    counter_mnemonics = r.json()['data']

    wait_until_SPECfinished()
    
    
    r = requests.get(bServer + "get_all_counters", params={})
    counter_values = r.json()['data']
   
    time_spent = time.time() - t0
    
    return dict(zip(counter_mnemonics, counter_values))


def move_to_ROSSindex(ross_index):
    r = requests.post("http://127.0.0.1:18080/combined_move/move_to_index/{}".format(ross_index))
    #print(r)
    #print(r.json())

In [78]:
g = read_counters()
print(g)

{'sec': 0.2, 'mon': 0.0004, 'det': 0.0, 'dead': 0.0, 'i1': 9e-05, 'i2': 0.00192, 'i3': 0.0, 'i4': 0.0, 'normlz': 0.0, 'fstat': 0.0, 'TEMP': 0.0, 'pd1': -4.0, 'pd2': 0.0, 'pd3': 0.0, 'pd4': 0.0, 'pd5': 0.0, 'pd6': 0.0, 'pd7': 0.0, 'pd8': 0.0, 'pd9': 0.0, 'pd10': 0.0, 'pd11': 0.0, 'pd12': 0.0, 'pd13': 0.0, 'pd14': 0.0, 'pd15': 0.0, 'pd16': 0.0, 'ICR': 0.0, 'OCR': 0.0, 'ROI1': 0.0, 'ROI2': 0.0, 'ROI3': 0.0, 'ROI4': 0.0, 'keith_I': 0.0, 'keith_V': 0.0, 'CTEMP': 0.0, 'timesec': 0.0, 'tempsec': 0.0, 'PD_MIN': 0.0, 'PD_MAX': 0.0, 'A34410': 0.0, 'agdaq1': 0.0, 'agdaq2': 0.0, 'agdaq3': 0.0, 'agdaq4': 0.0, 'CTEMPB': 0.0}


In [16]:
cam_refresh()

45.07417416572571

In [23]:
move_SPECmotor('spr3', 51.1)

0.28101372718811035

# Lets do some real work.... starting with the parameters defining the run.

Define the run name

In [79]:
scan_base_name = 'TESTforRossinsitu_C5andC2_1'
path_to_detector = ""
path_to_saved_data = ""

do_cam_refresh = True

Define the sample locations

In [74]:
sample_motors = {'x': 'spr2', 'y': 'spr3'}

sample_ROSSindex = {'QR1': "1", 'QR3': "0", 'air': "2"}
sample_global_offsets = {'QR1': {'x':47, 'y':50}, 'QR3': {'x':47, 'y':51}, 'air': {'x':49, 'y':52}}


exposure_time = 0.2
cam_refresh_freq = 5 #Number of scans before calling a cam refresh

Generate the positions to scan for each sample

In [69]:
sample_positions = {'QR1': [], 'QR3': [], 'air': []}


for rel_x_pos in np.linspace(0, 5, 5):
    for rel_y_pos in np.linspace(0, 5, 5):
        sample_positions['QR1'].append({'x': rel_x_pos + sample_global_offsets['QR1']['x'], 'y': rel_y_pos + sample_global_offsets['QR1']['y']})
        sample_positions['QR3'].append({'x': rel_x_pos + sample_global_offsets['QR3']['x'], 'y': rel_y_pos + + sample_global_offsets['QR3']['y']})
        
sample_positions['air'].append({'x': 0 + sample_global_offsets['air']['x'], 'y': 0 + sample_global_offsets['air']['y']})


#Rebuild the sample_scan_list. This is a bit more flexible way of constructing scans (e.g., you can interleave the samples or scan over positions in the sample... up to you)

sample_scan_list = []
sample_scan_list.append( {'QR1': sample_positions['QR1']})
sample_scan_list.append( {'QR3': sample_positions['QR3']})
sample_scan_list.append( {'air': sample_positions['air']})

print(sample_scan_list)

[{'QR1': [{'x': 47.0, 'y': 50.0}, {'x': 47.0, 'y': 51.25}, {'x': 47.0, 'y': 52.5}, {'x': 47.0, 'y': 53.75}, {'x': 47.0, 'y': 55.0}, {'x': 48.25, 'y': 50.0}, {'x': 48.25, 'y': 51.25}, {'x': 48.25, 'y': 52.5}, {'x': 48.25, 'y': 53.75}, {'x': 48.25, 'y': 55.0}, {'x': 49.5, 'y': 50.0}, {'x': 49.5, 'y': 51.25}, {'x': 49.5, 'y': 52.5}, {'x': 49.5, 'y': 53.75}, {'x': 49.5, 'y': 55.0}, {'x': 50.75, 'y': 50.0}, {'x': 50.75, 'y': 51.25}, {'x': 50.75, 'y': 52.5}, {'x': 50.75, 'y': 53.75}, {'x': 50.75, 'y': 55.0}, {'x': 52.0, 'y': 50.0}, {'x': 52.0, 'y': 51.25}, {'x': 52.0, 'y': 52.5}, {'x': 52.0, 'y': 53.75}, {'x': 52.0, 'y': 55.0}]}, {'QR3': [{'x': 47.0, 'y': 51.0}, {'x': 47.0, 'y': 52.25}, {'x': 47.0, 'y': 53.5}, {'x': 47.0, 'y': 54.75}, {'x': 47.0, 'y': 56.0}, {'x': 48.25, 'y': 51.0}, {'x': 48.25, 'y': 52.25}, {'x': 48.25, 'y': 53.5}, {'x': 48.25, 'y': 54.75}, {'x': 48.25, 'y': 56.0}, {'x': 49.5, 'y': 51.0}, {'x': 49.5, 'y': 52.25}, {'x': 49.5, 'y': 53.5}, {'x': 49.5, 'y': 54.75}, {'x': 49.5, 

Here we interleave some of the measurements between the two batteries to improve time resolution.

In [80]:
sample_positions = {'QR1': [], 'QR3': [], 'air': []}

sample_scan_list = []

for rel_x_pos in np.linspace(0, 5, 5):
    
    sample_positions['QR1'] = []
    sample_positions['QR3'] = []
    
    for rel_y_pos in np.linspace(0, 5, 10):
        sample_positions['QR1'].append({'x': rel_x_pos + sample_global_offsets['QR1']['x'], 'y': rel_y_pos + sample_global_offsets['QR1']['y']})
        sample_positions['QR3'].append({'x': rel_x_pos + sample_global_offsets['QR3']['x'], 'y': rel_y_pos + + sample_global_offsets['QR3']['y']})
        
    sample_scan_list.append( {'QR1': sample_positions['QR1']})
    sample_scan_list.append( {'QR3': sample_positions['QR3']})
    

sample_positions['air'].append({'x': 0 + sample_global_offsets['air']['x'], 'y': 0 + sample_global_offsets['air']['y']})

sample_scan_list.append( {'air': sample_positions['air']})

print(sample_scan_list)

[{'QR1': [{'x': 47.0, 'y': 50.0}, {'x': 47.0, 'y': 50.55555555555556}, {'x': 47.0, 'y': 51.111111111111114}, {'x': 47.0, 'y': 51.666666666666664}, {'x': 47.0, 'y': 52.22222222222222}, {'x': 47.0, 'y': 52.77777777777778}, {'x': 47.0, 'y': 53.333333333333336}, {'x': 47.0, 'y': 53.888888888888886}, {'x': 47.0, 'y': 54.44444444444444}, {'x': 47.0, 'y': 55.0}]}, {'QR3': [{'x': 47.0, 'y': 51.0}, {'x': 47.0, 'y': 51.55555555555556}, {'x': 47.0, 'y': 52.111111111111114}, {'x': 47.0, 'y': 52.666666666666664}, {'x': 47.0, 'y': 53.22222222222222}, {'x': 47.0, 'y': 53.77777777777778}, {'x': 47.0, 'y': 54.333333333333336}, {'x': 47.0, 'y': 54.888888888888886}, {'x': 47.0, 'y': 55.44444444444444}, {'x': 47.0, 'y': 56.0}]}, {'QR1': [{'x': 48.25, 'y': 50.0}, {'x': 48.25, 'y': 50.55555555555556}, {'x': 48.25, 'y': 51.111111111111114}, {'x': 48.25, 'y': 51.666666666666664}, {'x': 48.25, 'y': 52.22222222222222}, {'x': 48.25, 'y': 52.77777777777778}, {'x': 48.25, 'y': 53.333333333333336}, {'x': 48.25, 'y'

Take control of SPEC

In [17]:
r = requests.get(bServer + "get_remote_control")
print(r.json())

{'help': 'Get the current plot. Not sure what this actually is', 'data': 'client in control.'}


## Ok. Lets do the actual scanning

In [81]:
#These previous values are used to prevent unnecessary motor movements in long scans. Set to None in the beginning to force the first move.
prev_ROSSindex = None
prev_x_pos = None
prev_y_pos = None

ordering_index = 0
do_cam_refresh = False
for scan_ctr in range(20000): #Set number scans to something large?
    tscan = time.time()
    
    #Check if we need to do a cam refresh
    if do_cam_refresh is True:
        if scan_ctr % cam_refresh_freq == 0: 
            print("Doing a cam refresh")
            cam_refresh()
    else:
        print("RE ENABLE CAM REFRESH checking!!!!!")    
    
    #Loop through each elemnt in the sample scan_list
    for single_scan_list in sample_scan_list:
        
        #Extract the sample name and the positions over which to scan
        for (sample_name, sample_pos_list) in single_scan_list.items():
            

            #Move to the right position on the ROSS switcher
            new_ROSSindex = sample_ROSSindex[sample_name]
            if prev_ROSSindex != new_ROSSindex:
                move_to_ROSSindex(new_ROSSindex)
                prev_ROSSindex = new_ROSSindex

            #Make a new SPEC file for each scan
            scan_SPECfilename = "{}_i{}_loop{}_{}".format(scan_base_name, ordering_index, scan_ctr, sample_name)
            ordering_index = ordering_index + 1
            print("Creating new SPEC file: {}".format(scan_SPECfilename))
            create_SPECnewfile(scan_SPECfilename)

            
            #Loop through the positions on the sample and take data
            pos_ctr = 0
            for xy_pos in sample_pos_list:

                #Compute the coordinates
                x_pos = xy_pos['x']
                y_pos = xy_pos['y']

                #Move the SPEC motors to the correct positions
                tSPECmove = time.time()
                
                #Check if we actually need to move x
                if prev_x_pos != x_pos:
                    move_SPECmotor(sample_motors['x'], x_pos)
                    prev_x_pos = x_pos

                #Check if we actually need to move y
                if prev_y_pos != y_pos:
                    move_SPECmotor(sample_motors['y'], y_pos)
                    prev_y_pos = y_pos

                print("{} pos {} ({},{}): move took {}s".format(sample_name, pos_ctr, x_pos, y_pos, time.time() - tSPECmove))



                #Take data
                f_name = "{}_{}_p{}_sc{}".format(scan_base_name, sample_name, pos_ctr, scan_ctr)
                take_pilatus_image(exposure_time)

                

                pos_ctr = pos_ctr + 1 #Increment the position counter

    print("Scan {} finished in {}s".format(scan_ctr, time.time() - tscan))
    time.sleep(240.0) #wait?


            
        
        
        
        

RE ENABLE CAM REFRESH checking!!!!!
Creating new SPEC file: TESTforRossinsitu_C5andC2_1_i0_loop0_QR1
QR1 pos 0 (47.0,50.0): move took 0.7989399433135986s
QR1 pos 1 (47.0,50.55555555555556): move took 0.1601855754852295s
QR1 pos 2 (47.0,51.111111111111114): move took 0.16075992584228516s
QR1 pos 3 (47.0,51.666666666666664): move took 0.15999054908752441s
QR1 pos 4 (47.0,52.22222222222222): move took 0.15042471885681152s
QR1 pos 5 (47.0,52.77777777777778): move took 0.15639138221740723s
QR1 pos 6 (47.0,53.333333333333336): move took 0.13997411727905273s
QR1 pos 7 (47.0,53.888888888888886): move took 0.15023541450500488s
QR1 pos 8 (47.0,54.44444444444444): move took 0.14997339248657227s
QR1 pos 9 (47.0,55.0): move took 0.1504535675048828s
Creating new SPEC file: TESTforRossinsitu_C5andC2_1_i1_loop0_QR3
QR3 pos 0 (47.0,51.0): move took 0.6047673225402832s
QR3 pos 1 (47.0,51.55555555555556): move took 0.16000652313232422s
QR3 pos 2 (47.0,52.111111111111114): move took 0.16000795364379883s
Q

KeyboardInterrupt: 

Release control of SPEC

In [ ]:
r = requests.get(bServer + "release_remote_control")
print(r.json())